# Model

In [ ]:
from sympy import *
from sympy.vector import Dot
from sympy.abc import i, j, k, l

class PFC_Vacancy_Model:
    def __init__(self):
        # Space
        self.d = Symbol('d')            # Dimension of the space (e.g., 2D or 3D)
        self.V = Symbol('V')            # Volume of the system

        # Standard PFC symbols
        self.rho = Symbol('rho')        # Atomic number density
        self.rho_0 = Symbol('rho_0')    # Reference atomic number density
        self.n = Symbol('n')            # PFC density
        self.n_0 = Symbol('n_0')        # Reference PFC density

        self.pos = Symbol('\\vec{r}')   # Position vector
        self.lattice_vector_j = Symbol('\\vec{q}_j')  # Lattice vector
        self.lattice_point_i = Symbol('\\vec{R}_i')  # Lattice point

        self.alpha = Symbol('\\alpha')

        self.N_sites = Symbol('N_{sites}')  # Number of lattice sites
        self.N_j = Symbol('N_{j}')      # Number of lattice vectors

        # 1-mode ansatz
        self.A_j = Symbol('A_j')        # Amplitude of the j-th mode
        self.n1 = Symbol('n^1')         # 1-mode PFC density

    def Density_DFT_to_PFC(self):
        """
        Return the expression that converts DFT density to PFC density.
        """
        return (self.rho - self.rho_0) / self.rho_0
    
    def Density_PFC_to_DFT (self):
        """
        Return the expression that converts PFC density to DFT density.
        """
        return self.rho_0 * (1 + self.n)

    def Ansatz_1_mode(self):
        """
        Return the expression for the 1-mode ansatz in PFC.
        """
        return self.n_0 + Sum(self.A_j * exp(I * Dot(self.lattice_vector_j, self.pos)), (j, 1, self.N_j))

    def Ansatz_Gaussian(self):
        """
        Return the Gaussian ansatz for the PFC density.
        """
        return 1 / self.rho_0 * (self.alpha / pi)**(self.d/2) * Sum(exp(-self.alpha * abs(self.pos - self.lattice_point_i)**2), (i, 1, self.N_sites))

    def Amplitude_From_Guassian(self):
        """
        Return the expression for the amplitude from the Gaussian ansatz.
        """
        return (1 + self.n_0) * exp(-self.lattice_vector_j**2 / (4 * self.alpha))

# Model Tests

In [ ]:
model = PFC_Vacancy_Model()
model.Density_PFC_to_DFT()

In [ ]:
model.Ansatz_1_mode()

In [ ]:
model.Ansatz_Gaussian()

In [ ]:
model.Amplitude_From_Guassian()

## Density conversion consistency check

In [ ]:
# numeric consistency
model = PFC_Vacancy_Model()
_rho_0 = 1.
_rho = 1.2
_n_calc = float(model.Density_DFT_to_PFC().subs({model.rho: _rho, model.rho_0: _rho_0}))
_rho_calc = float(model.Density_PFC_to_DFT().subs({model.n: _n, model.rho_0: _rho_0}))

assert abs(_n_calc - (_rho - _rho_0) / _rho_0) < 1e-10, "Density conversion from DFT to PFC failed"
print(_rho, _rho_calc, _rho - _rho_calc) # should be close to each other

In [ ]:
# algebraic consistency
assert model.Density_PFC_to_DFT().subs({model.n: model.Density_DFT_to_PFC()}).simplify() == model.rho, "Algebraic consistency check failed"
assert model.Density_DFT_to_PFC().subs({model.rho: model.Density_PFC_to_DFT()}).simplify() == model.n, "Algebraic consistency check failed"

# Debug/tmp

In [ ]:
from sympy.vector import CoordSys3D, Vector
N = CoordSys3D('N')

v1, v2, v3 = symbols('v1 v2 v3', cls=Function)

In [ ]:
from sympy import symbols
from sympy.vector import CoordSys3D, Dot

# # Define a coordinate system (you don't have to use it explicitly, but it's useful)
# N = CoordSys3D('N')

# Define abstract vectors
a, b, c = symbols('a b c', real=True)

# Express your dot product operation
expr = Dot(a, b + c) - Dot(a, b)

# Simplify the expression
simplified_expr = expr.simplify()

simplified_expr

In [ ]:
r, tau, v, Bx = symbols('r \\tau v \\lambda')
epsilon = (r - 2 * tau - 3 * v) * v**(-Rational(1,3))
g = (tau + 3 * v) * v**(-Rational(2,3))

In [ ]:
epsilon

In [ ]:
g

In [ ]:
d, q, q0, A0, n0, phi0, beta, epsilon, g = symbols('d q q_0 A_0 n_0 \phi_0 \\beta \\epsilon g', real=True)

In [ ]:
q2p = Rational(1,2) * (q0**2 + sqrt(q0**4 - d * (1 + n0) / (3 * Bx * A0**2)))
q2p = q2p.subs({n0: phi0 - 1, Bx: beta * v**(Rational(1,3))}).simplify()
q2p

In [ ]:
q2m = Rational(1,2) * (q0**2 - sqrt(q0**4 - d * (1 + n0) / (3 * Bx * A0**2)))
q2m = q2m.subs({n0: phi0 - 1, Bx: beta * v**(Rational(1,3))}).simplify()
q2m

In [ ]:
zero = ln((1 + n0) / A0) * (3 * (-r - 2 * tau * n0 + 3 * v * n0**2 * A0**2 + Bx * (q**2 - q0**2)) * A0**2 + 6 * (-tau + 3 * v * n0) * A0**3 + 45 * v * A0**4) + d/4 * (1 + n0)
zero

In [ ]:
zero = zero.subs({n0: phi0 - 1, Bx: beta * v**(Rational(1,3)), r: -(3 - epsilon - 2*g), tau: -(3-g)}).simplify()
zero

In [ ]:
zerop = zero.subs({q**2: q2p}).simplify()
zerop

In [ ]:
zerom = zero.subs({q**2: q2m}).simplify()
zerom

In [ ]:
q2p = q2p.subs({d: 2, q0: 1, v: 1}).simplify()
q2p

In [ ]:
q2m = q2m.subs({d: 2, q0: 1, v: 1}).simplify()
q2m

In [ ]:
zerop = zerop.subs({d: 2, q0: 1, v: 1}).simplify()
zerop

In [ ]:
zerom = zerom.subs({d: 2, q0: 1, v: 1}).simplify()
zerom

In [ ]:
q2p.subs({beta: 2, epsilon: -3, g: -1})

In [ ]:
q2m.subs({beta: 2, epsilon: -3, g: -1})

In [ ]:
zerop.subs({beta: 2, epsilon: -3, g: -1})

In [ ]:
zerom.subs({beta: 2, epsilon: -3, g: -1})

In [ ]:
phi0_value = 0.856
A0p_sol = float(nsolve(2*zerop.subs({phi0: phi0_value, beta: 2, epsilon: -3, g: -1}), A0, 1))
A0m_sol = float(nsolve(2*zerom.subs({phi0: phi0_value, beta: 2, epsilon: -3, g: -1}), A0, 1))
q2p_sol = float(q2p.subs({phi0: phi0_value, beta: 2, epsilon: -3, g: -1, A0: A0p_sol}))
q2m_sol = float(q2m.subs({phi0: phi0_value, beta: 2, epsilon: -3, g: -1, A0: A0m_sol}))
A0p_sol, A0m_sol, q2p_sol, q2m_sol